In [ ]:
import json
import os
import subprocess
from IPython.display import clear_output

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device is", device)

device is cuda


# Generate metrics

In [34]:
from functions_utils_pomo import get_metrics     

In [35]:
cur_path = os.getcwd()
metrics = get_metrics(cur_path)
os.chdir('./Reward_weights')
for m in metrics:

    dic_tarif_sent_disp = {'v_required': 0,
                    'v_sent': 0,
                    'v_sent_full':0,
                    'v_degraded':0,
                    'cancelled':0, #cancel departure
                    'function_not_found':0,
                    'v1_not_sent_from_s1':0,
                    'v3_not_sent_from_s3':0,
                    'v_not_found_in_last_station':0,
                    'ff_required':0,
                    'ff_sent':0,
                    'rupture_ff':0,       
                    'z1_VSAV_sent': 0,
                    'z1_FPT_sent': 0,
                    'z1_EPA_sent': 0,
                     'VSAV_needed':0,
                     'FPT_needed':0,
                     'EPA_needed':0,
                     'VSAV_disp':0,
                     'FPT_disp':0,
                     'EPA_disp':0,
                    'skill_lvl':0
                    } 

    dic_tarif_sent_disp[m] = -100

    if m == 'v_degraded':
        
        dic_tarif_sent_disp['v_sent_full'] = 10


    with open(f"rw_"+ m +"_r100_cf3.json", "w") as f:
        json.dump(dic_tarif_sent_disp, f)

os.chdir("../")

In [36]:
os.chdir('./Reward_weights')
metric = "v1_not_sent_from_1st_station"
with open(f"rw_{metric}_r100_cf3.json", "r") as f:
    d = json.load(f)
os.chdir("../")
d

{'v_sent': 0,
 'v_sent_full': 0,
 'v_degraded': 0,
 'cancelled': 0,
 'function_not_found': 0,
 'v1_not_sent_from_1st_station': -100,
 'v_not_found_in_last_station': 0,
 'z1_sent': 0,
 'skill_lvl': 0}

# Agent params

In [38]:
path = "./SVG_model/shared_state_pomo.pt"
data = torch.load(path, map_location=device,weights_only=False)
print("Type :", type(data))
print(data.shape)
print(data)

Type : <class 'numpy.ndarray'>
(82, 40)
[[0.92307692 0.72727273 1.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 [0.125      0.125      0.125      ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [47]:
for metric in metrics:
    file_path = f"Reward_weights/rw_pomo_agent_{metric}_r100_cf3.json"
    try: os.remove(file_path)
    except FileNotFoundError: print(f"File '{file_path}' not found.")
print()

#-------Start
intervalle = 40
start = 1

while start <= 53080:
    
    end = start + intervalle - 1
    if end > 53080: end = 53080
    cmd = ["python3", "-u", "agent_pomo_run.py", "--start", str(start), "--end", str(end)]

    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )

    line_count = 0
    for line in process.stdout:
        line_count += 1
        #if line_count % 100 == 0:
           #clear_output(wait=True)

        print(line.strip())
    
    process.wait()
    
    start += intervalle
    

File 'Reward_weights/rw_pomo_agent_v3_not_sent_from_s3_r100_cf3.json' not found.
File 'Reward_weights/rw_pomo_agent_v_not_found_in_last_station_r100_cf3.json' not found.
File 'Reward_weights/rw_pomo_agent_z1_VSAV_sent_r100_cf3.json' not found.
File 'Reward_weights/rw_pomo_agent_rupture_ff_r100_cf3.json' not found.

Nombre d'agents = 2. On démarre...
[RANDOM] Agent élu: v1_not_sent_from_s1
[INFO] Fichier de reward de base chargé : rw_v1_not_sent_from_s1_r100_cf3.json
[INFO] Fichier de reward de base chargé : rw_v_degraded_r100_cf3.json
max_train_steps 195
lr decay: 1 decay_update: 100 PER 1
max_train_steps 195
lr decay: 1 decay_update: 100 PER 1
Dueling_QNetwork(
(model): Sequential(
(0): Linear(in_features=3280, out_features=1024, bias=True)
(1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
(2): ReLU()
(3): Linear(in_features=1024, out_features=1024, bias=True)
(4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
(5

KeyboardInterrupt: 

In [ ]:
import numpy as np

def load_agent_rewards(metrics, num_agents=2):
    """Charge les récompenses de tous les agents depuis les fichiers .npy"""
    all_rewards = []
    for i in range(num_agents):
        file_path = f"Plots/agent_pomo_metrics_{metrics[i]}_r100_cf3_train_reward_fake.npy"
        if os.path.exists(file_path):
            rewards = np.load(file_path)
            all_rewards.append(rewards)
    return np.array(all_rewards)

def compute_pomo_loss(all_rewards):
    """Calcule la loss POMO avec baseline partagée"""
    rewards_tensor = torch.tensor(all_rewards, dtype=torch.float32)
    baseline = rewards_tensor.mean(dim=0, keepdim=True)
    advantages = rewards_tensor - baseline
    loss = -torch.mean(advantages * torch.log(rewards_tensor + 1e-10))
    return loss

metrics = ['z1_sent', 'v_not_found_in_last_station']
num_agents = len(metrics)
all_rewards = load_agent_rewards(metrics, num_agents)
loss = compute_pomo_loss(all_rewards)

print(all_rewards)

%run plot_evo.py agent_pomo_metrics_z1_sent_r100_cf3_train_reward_fake.npy --interpolation 100000


[]


Exception: File `'plot_evo.py'` not found.